In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!git clone https://github.com/vandahalasi/BRAIN2SPEECH_LesssGoo.git

fatal: destination path 'BRAIN2SPEECH_LesssGoo' already exists and is not an empty directory.


In [3]:
%cd BRAIN2SPEECH_LesssGoo
!git pull
!ls

/content/BRAIN2SPEECH_LesssGoo
Already up to date.
'1D-CNN training.ipynb'			  requirements.txt
 create_dataloaders.py			  setup.py
 data_discovery_helpers			  SingleWordProductionDutch
 data_discovery_pre_processing.ipynb	  spectogram_dataset.py
 LSTM.py				  training_1participant.ipynb
 LSTM_Train_n_Validation_notebook.ipynb   train_stats.json
 README.md


In [4]:
!git submodule init
!git submodule update

In [5]:
%cd SingleWordProductionDutch
from reconstruction_minimal import createAudio
%cd ..

/content/BRAIN2SPEECH_LesssGoo/SingleWordProductionDutch
/content/BRAIN2SPEECH_LesssGoo


In [6]:
!pip install ray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
#import LSTM
#from create_dataloaders import get_data, create_datasets, create_dataloaders, write_statistics_to_json
from torch.optim import Adam
from torch.nn import MSELoss
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from functools import partial

In [8]:
%cd ..
!ls

/content
BRAIN2SPEECH_LesssGoo  drive  sample_data


# Utilities

In [9]:
import os

import numpy as np
import json
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

class SpectogramDataset(Dataset):   
    def __init__(self, data, spectogram, window, json_path):
        self.target_spectogram = np.array(spectogram, dtype=np.float32)
        self.features = data
        self.window = window
        
        # Opening JSON file for mean and std of training set
        f = open(json_path)
        stats = json.load(f)
        self.train_mean = stats["mean"]
        self.train_std = np.array(stats["std"])
        

    def __len__(self):
        return self.features.shape[0]-self.window


    def __getitem__(self, index):
        # Normalization
        input = (self.features[index:index+self.window,:]-self.train_mean)/(self.train_std + 1e-7)
        return input.astype(np.float32), self.target_spectogram[index:index+self.window,:]

def write_statistics_to_json(data):
    """
    Saving mean and std of training data into a json file for normalization.
    """
    stats = {}
    mean = np.mean(data,axis=0)
    std=np.std(data,axis=0)
    stats["mean"] = mean.tolist()
    stats["std"] = std.tolist()
    # Serializing json
    json_object = json.dumps(stats, indent=4)
    
    # Writing to sample.json
    with open("train_stats.json", "w") as outfile:
        outfile.write(json_object)


def get_data(feat_path, list_ptcp):
    """
    Loads features and spectograms into SpectogramDataset and creates train, 
    validation and test dataloaders.
    """

    participants = np.array(['sub-%02d'%i for i in range(1,11)])

    feat_names = np.load(os.path.join(feat_path,f'{participants[0]}_feat_names.npy'))
    feat_name_dict = {feat : idx for idx, feat in enumerate(feat_names)}
    for ptcp in participants:
        feat_names = np.load(os.path.join(feat_path,f'{ptcp}_feat_names.npy'))
        for feat_name in feat_names:
            if feat_name not in feat_name_dict.keys():
                feat_name_dict[feat_name] = len(feat_name_dict)

    for idx, ptcp in enumerate(participants[list_ptcp]):
        spec_ptcp = np.load(os.path.join(feat_path,f'{ptcp}_spec.npy'))
        feat_ptcp = np.load(os.path.join(feat_path,f'{ptcp}_feat.npy'))
        feat_names = np.load(os.path.join(feat_path,f'{ptcp}_feat_names.npy'))
        right_dim_feat = np.zeros((len(feat_ptcp),len(feat_name_dict)))
        for i, feat_name in enumerate(feat_names):
            j = feat_name_dict[feat_name]
            right_dim_feat[:,j] = feat_ptcp[:,i]

        train_spec, val_spec, test_spec = np.split(spec_ptcp, [int(len(spec_ptcp)*0.8), 
                int(len(spec_ptcp)*0.85)])
        train_feat, val_feat, test_feat = np.split(right_dim_feat, [int(len(right_dim_feat)*0.8), 
                int(len(right_dim_feat)*0.85)])

        if idx == 0:
            spectogram_train = train_spec
            spectogram_val = val_spec
            spectogram_test = test_spec
            features_train = train_feat
            features_val = val_feat
            features_test = test_feat

        spectogram_train = np.concatenate((spectogram_train, train_spec), axis=0)
        spectogram_val = np.concatenate((spectogram_val, val_spec), axis=0)
        spectogram_test = np.concatenate((spectogram_test, test_spec), axis=0)
        features_train = np.concatenate((features_train, train_feat), axis=0)
        features_val = np.concatenate((features_val, val_feat), axis=0)
        features_test = np.concatenate((features_test, test_feat), axis=0)

    return spectogram_train, spectogram_val, spectogram_test, features_train, features_val, features_test

def create_datasets(spectogram_train, spectogram_val, spectogram_test, features_train, features_val, features_test, json_path, 
        window=3):
    #create a Dataset
    train_dataset = SpectogramDataset(features_train, spectogram_train, window, json_path)
    val_dataset = SpectogramDataset(features_val, spectogram_val, window, json_path)
    test_dataset = SpectogramDataset(features_test, spectogram_test, window, json_path)

    return train_dataset, val_dataset, test_dataset

def create_dataloaders(train_dataset, val_dataset, test_dataset, batch_size=32):
    #create dataloader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle = True)
    eval_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle = False)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle = False)

    return train_loader, eval_loader, test_loader


# Model

In [10]:
import torch
import torch.nn as nn

class SpectogramLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=2, batch_first=True):
        super(SpectogramLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first)
        self.input_size=input_size
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.linear_projection = nn.Conv1d(in_channels=self.hidden_size, 
                out_channels=23, kernel_size=1)
        
    def forward(self, input_t):
        batch_size = input_t.size(0)
        window_size = input_t.size(1)
        feature_size = input_t.size(2)
        h0 = torch.zeros((batch_size, self.num_layers, self.hidden_size), dtype=torch.float32)
        c0 = torch.zeros((batch_size, self.num_layers, self.hidden_size), dtype=torch.float32)

        output, (hn, cn) = self.lstm(input_t)
        

        return self.linear_projection(output.permute(0,2,1)).permute(0,2,1)

### Create dataloaders:

In [15]:
import gc
gc.collect()

0

In [11]:
def load_data(window_size=32,batch_size=32,participant_list=[1]):
  JSON_PATH="/content/BRAIN2SPEECH_LesssGoo/train_stats.json"
  participant_list=[1]
  spectogram_train, spectogram_val, spectogram_test, features_train, features_val, features_test = get_data("/content/drive/MyDrive/BRAIN2SPEECH/features",participant_list)
 
  if(not(os.path.isfile(JSON_PATH))):
    write_statistics_to_json(features_train)

  train_dataset, val_dataset, test_dataset = create_datasets(spectogram_train, spectogram_val, 
          spectogram_test, features_train, features_val, features_test, json_path=JSON_PATH, window=window_size)
  train_loader, val_loader, test_loader = create_dataloaders(train_dataset, val_dataset, 
          test_dataset, batch_size=batch_size)
  
  return train_loader,val_loader,test_loader

In [12]:
train_loader, val_loader, _ = load_data()

## Training function

In [13]:
def train(config,checkpoint_dir):
    sum_loss = 0
    val_loss = 0

    # constant variables:
    EPOCH = 2
    JSON_PATH = r"BRAIN2SPEECH_LesssGoo/train_stats.json"
    PARTICIPANT_LIST = [1]

    # Tuneable variables:
    criterion = config['criterion']()
    model = SpectogramLSTM(input_size=4860, hidden_size=config['hidden_size'])
    optimizer = config['optimizer'](model.parameters(),lr=config['lr'])
    
    #writer:
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    writer = SummaryWriter(f"/content/drive/MyDrive/BRAIN2SPEECH/runs/E{EPOCH}_BS{config['batch_size']}_WS{config['window_size']}_PTCP{PARTICIPANT_LIST}_{format(timestamp)}")   

    # training
    print("training") 
    for epoch in range(EPOCH):
        model.train()
        sum_loss = 0
        print('EPOCH {}:'.format(epoch + 1))
        for i, (in_t, target) in tqdm(enumerate(train_loader)):
            in_t=in_t.cuda()
            target=target.cuda()
            outputs = model.forward(in_t)
            optimizer.zero_grad()
            
            loss = criterion(outputs, target)
            loss.backward()
            
            optimizer.step()

            sum_loss += loss.item()

            log_frequency = 99
            if i % log_frequency == 0:
                last_loss = sum_loss / log_frequency
                print('  batch {} loss: {}'.format(i, last_loss))
                summary_idx = epoch * len(train_loader) + i
                writer.add_scalar('Loss/train', last_loss, summary_idx)
                writer.flush()
                sum_loss = 0.0

        model.eval()
        val_loss=0.0
        val_steps = 0
        for in_v, target_v in val_loader:
            in_v=in_v.cuda()
            target_v=target_v.cuda()

            with torch.no_grad():
                outputs = model.forward(in_v)

            v_loss = criterion(outputs, target_v)
            val_loss += v_loss.item()
            val_steps += 1
        
        avg_vloss = val_loss / len(val_loader)
        print('LOSS valid {}'.format(avg_vloss))

        writer.add_scalar('Loss/Validation', avg_vloss, epoch + 1)
        writer.flush()

        with tune.checkpoint_dir(epoch) as checkpoint_dir:
          torch.save((model.state_dict(), optimizer.state_dict()), checkpoint_dir)

        tune.report(loss=(val_loss / val_steps))
    print("Finished Training")

In [14]:
def main(num_samples=10, max_num_epochs=1, gpus_per_trial=2):
    
    config = {
      "lr": tune.loguniform(1e-4, 1e-1),
      "window_size": tune.choice([16,32,64]),
      "batch_size": tune.choice([32,64,128]),
      "hidden_size": tune.choice([4860]),
      "criterion": tune.choice([MSELoss]),
      "optimizer": tune.choice([Adam])
    }
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    reporter = CLIReporter(
        parameter_columns=["lr", "window_size", "batch_size", "hidden_size","criterion","optimizer"],
        metric_columns=["loss", "training_iteration"])
   
    result = tune.run(
        partial(train),
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter)

    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))

    #test_acc = test_accuracy(best_trained_model, device)
    #print("Best trial test set accuracy: {}".format(test_acc))

In [5]:
main()

NameError: ignored

In [4]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive
